In [1]:
import cohere
import os
import uuid
from typing import List, Dict

co = cohere.Client("COHERE_API_KEY")

### Chatbot component

In [2]:
class Chatbot:
    def __init__(self, connectors: List[str]):
        self.conversation_id = str(uuid.uuid4())
        self.connectors = [{"id": c} for c in connectors]
        

    def generate_response(self, message: str):
        response = co.chat(
                        message=message,
                        connectors=self.connectors,
                        conversation_id=self.conversation_id,
                        stream=True
                        )

        for event in response:
            yield event
        yield response

### App component

In [4]:
class App:
    def __init__(self, chatbot: Chatbot):
        self.chatbot = chatbot
        
    def run(self):
        while True:
            # Get the user message
            message = input("User: ")

            # Typing "quit" ends the conversation
            if message.lower() == "quit":
                print("Ending chat.")
                break
            else:
                print(f"User: {message}")

            # Get the chatbot response
            response = self.chatbot.generate_response(message)

            # Print the chatbot response
            print("Chatbot:")
            
            citations_flag = False
            
            for event in response:
                stream_type = type(event).__name__
                
                # Text
                if stream_type == "StreamTextGeneration":
                    print(event.text, end="")

                # Citations
                if stream_type == "StreamCitationGeneration":
                    if not citations_flag:
                        print("\n\nCITATIONS:")
                        citations_flag = True
                    print(event.citations[0])
                
                # Documents
                if citations_flag:
                    if stream_type == "StreamingChat":
                        print("\n\nDOCUMENTS:")
                        documents = [{'id': doc['id'],
                                      'text': doc.get('text', doc.get('snippet', ''))[:50] + '...', # snippet field to account for web search results
                                      'title': doc['title'],
                                      'url': doc['url']} 
                                      for doc in event.documents]
                        for doc in documents:
                            print(doc)

            print(f"\n{'-'*100}\n")

### Run chatbot with Google Drive connector

In [5]:
# Define connectors
connectors = ["demo-conn-gdrive-6bfrp6"]

# Create an instance of the Chatbot class by supplying the connectors
chatbot = Chatbot(connectors)

# Create an instance of the App class with the Chatbot instance
app = App(chatbot)

# Run the chatbot
app.run()

User: What are sentence embeddings
Chatbot:
Sentence embeddings are a useful tool for validating output from an LLM. They can be used to ensure the output is similar enough to a target, for example, in text summarization tasks. Sentence embeddings can also be used to ensure the output meets certain criteria such as safety and correctness. For example, we may want to confirm that an output does not contain profanity.

CITATIONS:
{'start': 42, 'end': 59, 'text': 'validating output', 'document_ids': ['demo-conn-gdrive-6bfrp6_1:5']}
{'start': 68, 'end': 72, 'text': 'LLM.', 'document_ids': ['demo-conn-gdrive-6bfrp6_1:5']}
{'start': 114, 'end': 140, 'text': 'similar enough to a target', 'document_ids': ['demo-conn-gdrive-6bfrp6_1:5', 'demo-conn-gdrive-6bfrp6_1:4']}
{'start': 158, 'end': 183, 'text': 'text summarization tasks.', 'document_ids': ['demo-conn-gdrive-6bfrp6_1:5']}
{'start': 273, 'end': 279, 'text': 'safety', 'document_ids': ['demo-conn-gdrive-6bfrp6_1:4']}
{'start': 365, 'end': 3

### Run chatbot with web search connector

In [5]:
# Define connectors
connectors = ["web-search"]

# Create an instance of the Chatbot class by supplying the connectors
chatbot = Chatbot(connectors)

# Create an instance of the App class with the Chatbot instance
app = App(chatbot)

# Run the chatbot
app.run()

User: What is LLM university
Chatbot:
LLM University (LLMU) is a set of comprehensive learning resources for anyone interested in natural language processing (NLP) or large language models (LLMs), from beginners to advanced learners. 

You can customise your learning path and the curriculum covers everything from the fundamentals of LLMs to the most advanced topics, including generative AI.

CITATIONS:
{'start': 0, 'end': 21, 'text': 'LLM University (LLMU)', 'document_ids': ['web-search_8:0', 'web-search_8:2', 'web-search_8:1', 'web-search_5:0', 'web-search_5:3']}
{'start': 34, 'end': 66, 'text': 'comprehensive learning resources', 'document_ids': ['web-search_8:0', 'web-search_8:2', 'web-search_8:1', 'web-search_5:0']}
{'start': 92, 'end': 125, 'text': 'natural language processing (NLP)', 'document_ids': ['web-search_8:0', 'web-search_8:1', 'web-search_5:0']}
{'start': 129, 'end': 157, 'text': 'large language models (LLMs)', 'document_ids': ['web-search_8:1', 'web-search_5:0']}
{'star

### Run chatbot with multiple connectors

In [6]:
# Define connectors
connectors = ["demo-conn-gdrive-6bfrp6", "web-search"]

# Create an instance of the Chatbot class by supplying the connectors
chatbot = Chatbot(connectors)

# Create an instance of the App class with the Chatbot instance
app = App(chatbot)

# Run the chatbot
app.run()

User: What is chain of thought prompting
Chatbot:
Chain of thought prompting is a technique that guides LLMs (language model) to follow a reasoning process when dealing with problematic questions. When a prompt asks a model to give a final answer to a multi-step problem, chain of thought prompting induces the model to decompose the problem into intermediate reasoning steps, leading to a correct final answer. This is done by showing the model a few examples where the step-by-step reasoning is clearly laid out. The model is then expected to follow that "chain of thought" reasoning and get to the correct answer.

This approach has been found to significantly enhance the ability of LLMs to tackle complex arithmetic and commonsense reasoning tasks.

CITATIONS:
{'start': 47, 'end': 58, 'text': 'guides LLMs', 'document_ids': ['web-search_4:1']}
{'start': 79, 'end': 105, 'text': 'follow a reasoning process', 'document_ids': ['web-search_4:1']}
{'start': 184, 'end': 196, 'text': 'final answer',